<a href="https://colab.research.google.com/github/Abishethvarman/FakeNewsDetection/blob/classifier/3.6-FakeNewsClassfier_CNN_20k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [2]:
%cd /mydrive/dataset/

/content/gdrive/My Drive/dataset


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [5]:
df=pd.read_csv('fake-news-category-PES.csv')

In [6]:
df

,corpus,category
0,JOHN CONYERS’ LAWYER Threatens ‘If This Nonsen...,Politics
1,"(CNN Student News) -- December 20, 2013 . In o...",Entertainment
2,"ATLANTA, Georgia (CNN) -- So much for Southern...",Entertainment
3,George Soros-Financed Groups Scheme to Stop Tr...,Politics
4,NOT HILLARY’S TURN: LIB PUBLICATIONS ARE SAYIN...,Politics
...,...,...
59995,hahahahaha we missed 123455323 goals haha,Sports
59996,By . Katherine Faulkner and Nick Fagge . PUBLI...,Entertainment
59997,Trump Attacks New Mexico’s GOP Governor After...,Politics
59998,Trump Once Again Demonstrates Why His Russia ...,Politics


In [7]:
# Assuming df is your DataFrame
category_mapping = {'Politics': 1, 'Entertainment': 2, 'Sports': 3}
df['label'] = df['category'].map(category_mapping)

In [8]:
df

,corpus,category,label
0,JOHN CONYERS’ LAWYER Threatens ‘If This Nonsen...,Politics,1
1,"(CNN Student News) -- December 20, 2013 . In o...",Entertainment,2
2,"ATLANTA, Georgia (CNN) -- So much for Southern...",Entertainment,2
3,George Soros-Financed Groups Scheme to Stop Tr...,Politics,1
4,NOT HILLARY’S TURN: LIB PUBLICATIONS ARE SAYIN...,Politics,1
...,...,...,...
59995,hahahahaha we missed 123455323 goals haha,Sports,3
59996,By . Katherine Faulkner and Nick Fagge . PUBLI...,Entertainment,2
59997,Trump Attacks New Mexico’s GOP Governor After...,Politics,1
59998,Trump Once Again Demonstrates Why His Russia ...,Politics,1


In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','would'])

In [10]:
def preprocess(text):
    result = []
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    if pd.notna(text):
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
                result.append(lemmatizer.lemmatize(token))

    return ' '.join(result)

In [11]:
df['clean_corpus'] = df['corpus'].apply(preprocess)

In [12]:
print(df['clean_corpus'][0])

john conyers lawyer threatens nonsense continues harbinger come video john conyers lawyer arnold reed threatened come press conference today lady gentleman promise nonsense continues harbinger come reed said defending congressman press conference detroit reed trying blame accuser conyers ready name marion brown staffer conyers revealed week accuser thursday interview today alleged conyers violated body frequently proposition barrister ball lawyer john conyersreed push reed pushed allegation showed signed statement conyers staffer stated witness harassment sexual misconduct congressman reed head urban dictionary said victim conyers mean kissing public relationship campbell indicates verified congressman hired accuser daughter problem hell broke loose congressman fired daughter sudden sexual harassment sexual allegation problem predator talking barrister ball barrister ball reed said held photograph appeared conyers brown posing picture brown said took opportunity hour harass animal barr

In [27]:
print(df['label'].unique())

[1 2 3]


In [28]:
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=4)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=4)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_corpus'], df['label'], test_size=0.2, random_state=42)

In [30]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [31]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [32]:
X_train_pad = pad_sequences(X_train_seq, maxlen=100)
X_test_pad = pad_sequences(X_test_seq, maxlen=100)

In [37]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=50, input_length=100))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 50)           250000    
                                                                 
 conv1d_2 (Conv1D)           (None, 96, 128)           32128     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                      

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
history = model.fit(X_train_pad, y_train, epochs=5, validation_data=(X_test_pad, y_test), batch_size=64)

Epoch 1/5
750/750 [==============================] - 29s 38ms/step - loss: 0.0000e+00 - accuracy: 0.3342 - val_loss: 0.0000e+00 - val_accuracy: 0.3298
Epoch 2/5
750/750 [==============================] - 32s 43ms/step - loss: 0.0000e+00 - accuracy: 0.3342 - val_loss: 0.0000e+00 - val_accuracy: 0.3298
Epoch 3/5
750/750 [==============================] - 30s 40ms/step - loss: 0.0000e+00 - accuracy: 0.3342 - val_loss: 0.0000e+00 - val_accuracy: 0.3298
Epoch 4/5
750/750 [==============================] - 42s 57ms/step - loss: 0.0000e+00 - accuracy: 0.3342 - val_loss: 0.0000e+00 - val_accuracy: 0.3298
Epoch 5/5
750/750 [==============================] - 40s 53ms/step - loss: 0.0000e+00 - accuracy: 0.3342 - val_loss: 0.0000e+00 - val_accuracy: 0.3298


In [40]:
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)

375/375 [==============================] - 2s 6ms/step


In [41]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.33


In [42]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.33      1.00      0.50      3958
           2       0.00      0.00      0.00      4020
           3       0.00      0.00      0.00      4022

    accuracy                           0.33     12000
   macro avg       0.11      0.33      0.17     12000
weighted avg       0.11      0.33      0.16     12000

Confusion Matrix:
[[3958    0    0]
 [4020    0    0]
 [4022    0    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
# phrase_1 = 'Henry Dunant came for a delegation in Italy to discuss about the guyana issue'
new_phrase = 'Tom cruise for a delegation in Italy to discuss about the guyana issue'

In [48]:
cleaned_new_phrase = preprocess(new_phrase)
new_phrase_seq = tokenizer.texts_to_sequences([cleaned_new_phrase])
new_phrase_pad = pad_sequences(new_phrase_seq, maxlen=100)

predicted_probabilities = model.predict(new_phrase_pad)
predicted_class = tf.argmax(predicted_probabilities, axis=1).numpy()[0] + 1

predicted_class_name = {1: 'Politics', 2: 'Entertainment', 3: 'Sports'}.get(predicted_class, 'Unknown')

print(f"The predicted class for the phrase '{new_phrase}' is: {predicted_class_name}")


1/1 [==============================] - 0s 65ms/step
The predicted class for the phrase 'Tom cruise for a delegation in Italy to discuss about the guyana issue' is: Politics


In [49]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.33
Precision: 0.11
Recall: 0.33
F1 Score: 0.16


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-------------------------------------